In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
# from config import API_KEY_INFO
# serviceKey1 = API_KEY_INFO.get("portal")
# serviceKey2 = API_KEY_INFO.get("portal2")

## Define Class

In [43]:
import requests
import pandas as pd

class Kbland:

    def __init__(self):
        self.월간주간구분코드 = {
            "01": "월간",
            "02": "주간",
        }
        self.매물종별구분 = {
            "01": "아파트",
            "08": "연립",
            "09": "단독",
            "98": "주택종합",
        }
        self.매매전세코드 = {
            "01": "매매",
            "02": "전세",
        }

    def get_price_index(self, 월간주간구분코드, 매물종별구분, 매매전세코드, **kwargs):
        """
        KB통계 - 주택가격동향조사 - 가격지수
        
        Parameters
        ----------
        월간주간구분코드 : str
            월간주간구분코드
            01: 월간
            02: 주간
        매물종별구분 : str
            매물종별구분
            01: 아파트
            08: 연립
            09: 단독
            98: 주택종합
        매매전세코드 : str
            매매전세코드
            01: 매매
            02: 전세
        **kwargs : dict
            그 외 필요한 파라미터
            지역코드 : str
        """
        url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/priceIndex"

        # 매물종별구분이 '01'이 아니면 월간주간구분코드는 '01'
        if 매물종별구분 != "01":
            월간주간구분코드 = "01"
        params = {
            "월간주간구분코드": 월간주간구분코드,
            "매물종별구분": 매물종별구분,
            "매매전세코드": 매매전세코드,
        }
        params.update(kwargs)
        try:
            res = requests.get(url, params=params)
            data = res.json()['dataBody']['data']
        except Exception as e:
            print(e)
            return
        result_code = res.json()['dataBody']['resultCode']
        if str(result_code) != "11000":
            print(data['message'])
            return 
        n_data_list = len(data['데이터리스트'][0]['dataList'])
        n_date_list = len(data['날짜리스트'])
        n_date_str = len(data['날짜리스트'][0])
        columns = data['날짜리스트']
        df = pd.DataFrame(data['데이터리스트'])
        values = pd.DataFrame(df['dataList'].values.tolist()).iloc[:, :n_date_list]
        values.columns = columns
        df = pd.concat([df[['지역코드','지역명']], values], axis=1)
        df2 = pd.melt(df, id_vars=['지역코드','지역명']).rename(columns={'variable':'날짜', 'value':'가격지수'})
        if n_date_str == 6:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m')
        elif n_date_str == 8:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m%d')
        df2 = df2.sort_values(['지역코드','날짜'], ascending=[True, True]).reset_index(drop=True)
        # 코드 정보 부여
        code_df = pd.DataFrame({
            "월간주간구분": [self.월간주간구분코드[월간주간구분코드]],
            "매물종별구분": [self.매물종별구분[매물종별구분]],
            "거래구분": [self.매매전세코드[매매전세코드]],
            })
        code_df = pd.concat([code_df] * len(df2), ignore_index=True)
        df2 = pd.concat([code_df, df2], axis=1)
        return df2

    def get_monthly_apartment_wolse_index(self, **kwargs):
        """
        KB통계 - 주택가격동향조사 - 월간 아파트 월세가격지수

        Parameters
        ----------
        **kwargs : dict
            선택
        """
        url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/mntlyAptMrpayPrcIndx"
        params = {}
        params.update(kwargs)
        try:
            res = requests.get(url, params=params)
            data = res.json()['dataBody']['data']
        except Exception as e:
            print(e)
            return
        result_code = res.json()['dataBody']['resultCode']
        if str(result_code) != "11000":
            print(data['message'])
            return 
        n_data_list = len(data['데이터리스트'][0]['dataList'])
        n_date_list = len(data['날짜리스트'])
        n_date_str = len(data['날짜리스트'][0])
        columns = data['날짜리스트']
        df = pd.DataFrame(data['데이터리스트'])
        values = pd.DataFrame(df['dataList'].values.tolist()).iloc[:, :n_date_list]
        values.columns = columns
        df = pd.concat([df[['지역코드','지역명']], values], axis=1)
        df2 = pd.melt(df, id_vars=['지역코드','지역명']).rename(columns={'variable':'날짜', 'value':'가격지수'})
        if n_date_str == 6:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m')
        elif n_date_str == 8:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m%d')
        df2 = df2.sort_values(['지역코드','날짜'], ascending=[True, True]).reset_index(drop=True)
        # 코드 정보 부여
        code_df = pd.DataFrame({
            "월간주간구분": ["월간"],
            "매물종별구분": ["아파트"],
            "거래구분": ["월세"],
            })
        code_df = pd.concat([code_df] * len(df2), ignore_index=True)
        df2 = pd.concat([code_df, df2], axis=1)
        return df2

    def get_lead_apartment_50_index(self, **kwargs):
        """
        KB통계 - 주택가격동향조사 - KB선도아파트 50 지수
        """
        url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/leadApt50Indx"
        params = {}
        params.update(kwargs)
        try:
            res = requests.get(url, params=params)
            data = res.json()['dataBody']['data']
        except Exception as e:
            print(e)
            return
        result_code = res.json()['dataBody']['resultCode']
        if str(result_code) != "11000":
            print(data['message'])
            return 
        n_data_list = len(data['선도50지수리스트'])
        n_date_list = len(data['날짜리스트'])
        n_date_str = len(data['날짜리스트'][0])
        columns = data['날짜리스트']

        df = pd.DataFrame({
            "날짜": data['날짜리스트'],
            "선도50지수": data['선도50지수리스트'],
            "전월대비증감률": data['전월대비증감률리스트'],
            "전년동월대비증감률": data['전년동월대비증감률리스트'],
            })

        if n_date_str == 6:
            df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m')
        elif n_date_str == 8:
            df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d')
        df = df.sort_values(['날짜'], ascending=[True]).reset_index(drop=True)
        return df

    def get_price_index_change_rate(self, 월간주간구분코드, 매물종별구분, 매매전세코드, **kwargs):
        """
        KB통계 - 주택가격동향조사 - 가격지수증감률

        Parameters
        ----------
        월간주간구분코드 : str
            월간주간구분코드
            01: 월간
            02: 주간
        매물종별구분 : str
            매물종별구분
            01: 아파트
            08: 연립
            09: 단독
            98: 주택종합
        매매전세코드 : str
            매매전세코드
            01: 매매
            02: 전세
        **kwargs : dict
            그 외 필요한 파라미터
        """
        url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/prcIndxInxrdcRt"
        # 매물종별구분이 '01'이 아니면 월간주간구분코드는 '01'
        if 매물종별구분 != "01":
            월간주간구분코드 = "01"
        params = {
            "월간주간구분코드": 월간주간구분코드,
            "매물종별구분": 매물종별구분,
            "매매전세코드": 매매전세코드,
        }
        params.update(kwargs)
        try:
            res = requests.get(url, params=params)
            data = res.json()['dataBody']['data']
        except Exception as e:
            print(e)
            return
        result_code = res.json()['dataBody']['resultCode']
        if str(result_code) != "11000":
            print(data['message'])
            return 
        n_data_list = len(data['데이터리스트'][0]['dataList'])
        n_date_list = len(data['날짜리스트'])
        n_date_str = len(data['날짜리스트'][0])
        columns = data['날짜리스트']
        df = pd.DataFrame(data['데이터리스트'])
        values = pd.DataFrame(df['dataList'].values.tolist()).iloc[:, :n_date_list]
        values.columns = columns
        df = pd.concat([df[['지역코드','지역명']], values], axis=1)
        df2 = pd.melt(df, id_vars=['지역코드','지역명']).rename(columns={'variable':'날짜', 'value':'가격지수증감률'})
        if n_date_str == 6:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m')
        elif n_date_str == 8:
            df2['날짜'] = pd.to_datetime(df2['날짜'], format='%Y%m%d')
        df2 = df2.sort_values(['지역코드','날짜'], ascending=[True, True]).reset_index(drop=True)
        # 코드 정보 부여
        code_df = pd.DataFrame({
            "월간주간구분": [self.월간주간구분코드[월간주간구분코드]],
            "매물종별구분": [self.매물종별구분[매물종별구분]],
            "거래구분": [self.매매전세코드[매매전세코드]],
            })
        code_df = pd.concat([code_df] * len(df2), ignore_index=True)
        df2 = pd.concat([code_df, df2], axis=1)
        return df2

In [44]:
# API 인스턴스 생성
api = Kbland()

In [46]:
params = {
    "월간주간구분코드": "02",
    "매물종별구분": "01",
    "매매전세코드": "01",
    # "지역코드": "41",
    "기간": 1,
}
df = api.get_price_index(**params)
df.loc[df['지역명']=='전국'].tail()

,월간주간구분,매물종별구분,거래구분,지역코드,지역명,날짜,가격지수
46,주간,아파트,매매,0000000000,전국,2022-11-21,97.670740
47,주간,아파트,매매,0000000000,전국,2022-11-28,97.354538
48,주간,아파트,매매,0000000000,전국,2022-12-05,96.888385
49,주간,아파트,매매,0000000000,전국,2022-12-12,96.569873
50,주간,아파트,매매,0000000000,전국,2022-12-19,96.149852


In [40]:
params = {
    "기간": 1,
}
df = api.get_monthly_apartment_wolse_index(**params)
df.tail()

,월간주간구분,매물종별구분,거래구분,지역코드,지역명,날짜,가격지수
73,월간,아파트,월세,4100000000,경기도,2022-08-01,105.904649
74,월간,아파트,월세,4100000000,경기도,2022-09-01,106.492218
75,월간,아파트,월세,4100000000,경기도,2022-10-01,107.347923
76,월간,아파트,월세,4100000000,경기도,2022-11-01,107.957964
77,월간,아파트,월세,4100000000,경기도,2022-12-01,107.956571


In [12]:
params = {
    "기간": 1,
}
df = api.get_lead_apartment_50_index(**params)
df.tail()

,날짜,선도50지수,전월대비증감률,전년동월대비증감률
8,2022-08-01,100.451199,-0.718366,6.258222
9,2022-09-01,99.322158,-1.123969,3.105370
10,2022-10-01,97.584598,-1.749418,-0.118113
11,2022-11-01,94.517070,-3.143456,-4.390270
12,2022-12-01,92.074998,-2.583736,-7.554604


In [41]:
params = {
    "월간주간구분코드": "01",
    "매물종별구분": "98",
    "매매전세코드": "01",
    "기간": 1,
}
df = api.get_price_index(**params)
df.loc[df['지역명']=='전국'].tail()

,월간주간구분,매물종별구분,거래구분,지역코드,지역명,날짜,가격지수
8,월간,주택종합,매매,0000000000,전국,2022-08-01,100.727588
9,월간,주택종합,매매,0000000000,전국,2022-09-01,100.567640
10,월간,주택종합,매매,0000000000,전국,2022-10-01,100.013697
11,월간,주택종합,매매,0000000000,전국,2022-11-01,98.908737
12,월간,주택종합,매매,0000000000,전국,2022-12-01,97.893770


In [42]:
params = {
    "월간주간구분코드": "01",
    "매물종별구분": "98",
    "매매전세코드": "01",
    "기간": 1,
}
df = api.get_price_index_change_rate(**params)
df.loc[df['지역명']=='전국'].tail()

,월간주간구분,매물종별구분,거래구분,지역코드,지역명,날짜,가격지수증감률
8,월간,주택종합,매매,0000000000,전국,2022-08-01,-0.140020
9,월간,주택종합,매매,0000000000,전국,2022-09-01,-0.158792
10,월간,주택종합,매매,0000000000,전국,2022-10-01,-0.550817
11,월간,주택종합,매매,0000000000,전국,2022-11-01,-1.104809
12,월간,주택종합,매매,0000000000,전국,2022-12-01,-1.026165


## Develop

In [ ]:
# 매매가격 대비 전세가격 비율 deal count and tenant ratio
# 매물종별구분
url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/dealCntstTnantRato"

# 전월세전환율
url = "https://data-api.kbland.kr/bfmstat/weekMnthlyHuseTrnd/jnmnrnCnvsnRt"